In [ ]:
import numpy as np
import random

In [1]:
def rwma(data):
    #data is an N by T array with returns for each expert/stock i in time t forming a row
    
    N = len(data[0]) # number of stocks
    T = len(data)
    
    eps = np.sqrt(np.log(N)/T)
    
    w = np.ones(N)
    #M = np.zeros(N)
    #M_wma_random = 0

    #allweights = []

    # iterations
    for t in range (T):
        
        # reveal outcome, suppose it is random
        y = (data[t]>0).astype(int) #find for each expert if return was positive or not
        
        #M_wma_random += (x != y) #count number of mistakes

        # weight update
        for i in range(N):
            w[i] = w[i]*(1 - eps)**(y[i]) # penalize weight if made wrong prediction
        #M = M + (f != y)
   
    return w